<a href="https://colab.research.google.com/github/RAFS20/RIGEL/blob/main/RIGEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
# RIGEL v1.1 — Robust Interpretable Generalized Ensemble Learner
# Autor: Ricardo Alonzo Fernández Salguero


import os, sys, math, json, time, textwrap, warnings, typing, inspect
from typing import Tuple, Dict, List
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

SEED = 123
RNG = np.random.default_rng(SEED)

# --------------------------------- Utilities ----------------------------------
def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

OUT_DIR = "/content/rigel_out"
ensure_dir(OUT_DIR)

def hdr(title: str):
    print("\n" + "="*90)
    print(title)
    print("="*90)

def env_report():
    import sklearn, numpy, pandas, matplotlib
    rep = {
        "python": sys.version.split()[0],
        "numpy": numpy.__version__,
        "pandas": pandas.__version__,
        "matplotlib": matplotlib.__version__,
        "sklearn": sklearn.__version__,
    }
    print("Environment:", rep)
    return rep

# --------------------------- Synthetic Data Generators ------------------------
def inject_missing_and_outliers(X: np.ndarray, missing_frac=0.05, outlier_frac=0.02, outlier_scale=10.0):
    X = X.copy()
    n, d = X.shape
    m_mask = RNG.random((n, d)) < missing_frac
    X[m_mask] = np.nan
    o_mask = RNG.random((n, d)) < outlier_frac
    X[o_mask] = X[o_mask] * RNG.uniform(0, outlier_scale, size=o_mask.sum())
    return X

def make_mixed_features(n: int, d_num: int, d_cat: int, seed=None):
    r = np.random.default_rng(seed)
    X_num = r.normal(size=(n, d_num))
    cats = []
    for j in range(d_cat):
        k = r.integers(2, 8)  # 2..7 categories
        z = r.normal(size=n)
        edges = np.quantile(z, np.linspace(0, 1, k+1)[1:-1])
        xj = np.digitize(z, edges)
        cats.append(xj.reshape(-1,1))
    X_cat = np.hstack(cats) if d_cat>0 else np.empty((n,0))
    X = np.hstack([X_num, X_cat])
    return X, d_num, d_cat

def gen_classification_dataset(n=3000, d_num=8, d_cat=4, kind="mixed", seed=None, imb=0.65):
    r = np.random.default_rng(seed)
    X, dnum, dcat = make_mixed_features(n, d_num, d_cat, seed=seed)
    num = X[:, :dnum] if dnum>0 else np.empty((n,0))
    cat = X[:, dnum:] if dcat>0 else np.empty((n,0))

    emb = []
    for j in range(dcat):
        k = int(cat[:, j].max()) + 1
        E = r.normal(size=(k, 3))
        emb.append(E[cat[:, j].astype(int)])
    Ecat = np.hstack(emb) if dcat>0 else np.zeros((n,0))

    if kind == "linear":
        z = (num @ r.normal(size=dnum)) + (Ecat @ r.normal(size=Ecat.shape[1]))
    elif kind == "step":
        z = np.where((num[:,0] + (num[:,1]**2 if dnum>1 else 0)) > 0, 2.0, -2.0) + (Ecat @ r.normal(size=Ecat.shape[1]))
    elif kind == "sinusoidal":
        z = np.sin(num[:,0]*2.5) + (np.cos(num[:,1]*1.3) if dnum>1 else 0) + (Ecat @ r.normal(size=Ecat.shape[1]))
    elif kind == "xor":
        a = (num[:,0] > 0).astype(int) if dnum>0 else r.integers(0,2,size=n)
        b = (num[:,1] > 0).astype(int) if dnum>1 else r.integers(0,2,size=n)
        z = np.where(a ^ b, 2.0, -2.0) + (Ecat @ r.normal(size=Ecat.shape[1]))
    else:  # mixed
        z = (
            0.8*np.sin(num[:,0]) +
            0.5*(num[:,1]**2 if dnum>1 else 0) +
            0.3*(num[:,2]*num[:,3] if dnum>3 else 0) +
            0.6*(Ecat @ r.normal(size=Ecat.shape[1]))
        )
    noise = r.normal(scale=1 + 0.5*np.abs(num[:,0]) if dnum>0 else 1, size=n)
    logits = z + 0.8*noise
    thr = np.quantile(logits, imb)
    y = (logits > thr).astype(int)

    X = inject_missing_and_outliers(X, missing_frac=0.05, outlier_frac=0.02, outlier_scale=12.)
    return X, y, dnum, dcat

def gen_regression_dataset(n=3000, d_num=8, d_cat=4, kind="mixed", seed=None):
    r = np.random.default_rng(seed)
    X, dnum, dcat = make_mixed_features(n, d_num, d_cat, seed=seed)
    num = X[:, :dnum] if dnum>0 else np.empty((n,0))
    cat = X[:, dnum:] if dcat>0 else np.empty((n,0))

    emb = []
    for j in range(dcat):
        k = int(cat[:, j].max()) + 1
        E = r.normal(size=(k, 3))
        emb.append(E[cat[:, j].astype(int)])
    Ecat = np.hstack(emb) if dcat>0 else np.zeros((n,0))

    if kind == "linear":
        y = (num @ r.normal(size=dnum)) + (Ecat @ r.normal(size=Ecat.shape[1]))
    elif kind == "step":
        y = np.where((num[:,0] + (num[:,1]**2 if dnum>1 else 0)) > 0, 3.0, -1.5) + (Ecat @ r.normal(size=Ecat.shape[1]))
    elif kind == "sinusoidal":
        y = 2*np.sin(num[:,0]*2.0) + (np.cos(num[:,1]*1.7) if dnum>1 else 0) + (Ecat @ r.normal(size=Ecat.shape[1]))
    elif kind == "hetero":
        base = (num[:,0] if dnum>0 else 0) + (Ecat @ r.normal(size=Ecat.shape[1]))
        sigma = 0.3 + 0.7*np.abs(np.sin(base))
        y = base + r.normal(scale=sigma)
    else:  # mixed
        y = (
            1.2*np.sin(num[:,0]) +
            0.7*(num[:,1]**2 if dnum>1 else 0) +
            0.4*(num[:,2]*num[:,3] if dnum>3 else 0) +
            0.8*(Ecat @ r.normal(size=Ecat.shape[1]))
        )
    y = y + r.normal(scale=0.5, size=y.shape[0])
    X = inject_missing_and_outliers(X, missing_frac=0.05, outlier_frac=0.02, outlier_scale=12.)
    return X, y, dnum, dcat

# ------------------------------ Models & Prep ---------------------------------
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, QuantileTransformer, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score, roc_auc_score, log_loss, brier_score_loss,
    r2_score, mean_squared_error, mean_absolute_error
)
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingClassifier, HistGradientBoostingRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.ensemble import StackingClassifier, StackingRegressor
from sklearn.calibration import CalibratedClassifierCV

# Optional external baselines
HAVE_XGB = False
try:
    import xgboost as xgb
    HAVE_XGB = True
except Exception:
    HAVE_XGB = False

def make_onehot_kwargs():
    # sklearn >=1.4 usa sparse_output; versiones previas usan sparse
    sig = inspect.signature(OneHotEncoder.__init__)
    if 'sparse_output' in sig.parameters:
        return dict(handle_unknown="ignore", sparse_output=False)
    else:
        return dict(handle_unknown="ignore", sparse=False)

def make_preprocessor(dnum, dcat, for_regression=False):
    num_idx = list(range(dnum))
    cat_idx = list(range(dnum, dnum+dcat))
    transformers = []
    if dnum>0:
        steps = [("imp", SimpleImputer(strategy="median")),
                 ("qt", QuantileTransformer(output_distribution="normal", subsample=10000))]
        if for_regression:
            steps.append(("pt", PowerTransformer(method="yeo-johnson")))
        transformers.append(("num", Pipeline(steps), num_idx))
    if dcat>0:
        oh_kwargs = make_onehot_kwargs()
        transformers.append(("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("oh", OneHotEncoder(**oh_kwargs))
        ]), cat_idx))
    return ColumnTransformer(transformers)

def make_baselines_classif(dnum, dcat):
    pre = make_preprocessor(dnum, dcat, for_regression=False)
    models = {
        "LogReg": Pipeline([("pre", pre), ("clf", LogisticRegression(max_iter=400))]),
        "RF": Pipeline([("pre", pre), ("clf", RandomForestClassifier(n_estimators=400, min_samples_leaf=2, n_jobs=-1))]),
        "HGB": Pipeline([("pre", pre), ("clf", HistGradientBoostingClassifier(max_depth=None))]),
        "MLP": Pipeline([("pre", pre), ("clf", MLPClassifier(hidden_layer_sizes=(96,48), max_iter=400))]),
    }
    if HAVE_XGB:
        models["XGB"] = Pipeline([("pre", pre), ("clf", xgb.XGBClassifier(
            n_estimators=600, max_depth=6, subsample=0.9, colsample_bytree=0.9,
            tree_method="hist", eval_metric="logloss", n_jobs=-1))])
    return models

def make_rigel_classif(dnum, dcat):
    pre = make_preprocessor(dnum, dcat, for_regression=False)
    estimators = [
        ("rf", Pipeline([("pre", pre), ("m", RandomForestClassifier(n_estimators=500, min_samples_leaf=2, n_jobs=-1))])),
        ("hgb", Pipeline([("pre", pre), ("m", HistGradientBoostingClassifier(max_depth=None))])),
        ("log", Pipeline([("pre", pre), ("m", LogisticRegression(max_iter=500))])),
        ("mlp", Pipeline([("pre", pre), ("m", MLPClassifier(hidden_layer_sizes=(96,48), max_iter=500))])),
    ]
    if HAVE_XGB:
        estimators.append(("xgb", Pipeline([("pre", pre), ("m", xgb.XGBClassifier(
            n_estimators=800, max_depth=6, subsample=0.9, colsample_bytree=0.9,
            tree_method="hist", eval_metric="logloss", n_jobs=-1))])))
    # n_jobs no siempre está en StackingClassifier; evitamos pasarlo para compatibilidad
    # Removed preprocessor from final_estimator
    stack = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(max_iter=600), passthrough=False, cv=5)
    rigel = CalibratedClassifierCV(stack, method="isotonic", cv=5)
    return rigel

def make_baselines_reg(dnum, dcat):
    pre = make_preprocessor(dnum, dcat, for_regression=True)
    models = {
        "Ridge": Pipeline([("pre", pre), ("reg", Ridge(alpha=1.0))]),
        "RF": Pipeline([("pre", pre), ("reg", RandomForestRegressor(n_estimators=500, min_samples_leaf=2, n_jobs=-1))]),
        "HGB": Pipeline([("pre", pre), ("reg", HistGradientBoostingRegressor(max_depth=None))]),
        "MLP": Pipeline([("pre", pre), ("reg", MLPRegressor(hidden_layer_sizes=(96,48), max_iter=600))]),
    }
    if HAVE_XGB:
        models["XGB"] = Pipeline([("pre", pre), ("reg", xgb.XGBRegressor(
            n_estimators=800, max_depth=6, subsample=0.9, colsample_bytree=0.9, tree_method="hist", n_jobs=-1))])
    return models

class RigelRegressor:
    """
    RIGEL++ Regressor:
      - Base learners: RF, HGB, Ridge, MLP (+ optional XGB)
      - Stacked with Ridge meta-learner (passthrough features)
      - Split-conformal intervals for marginal coverage (90% by default)
    """
    def __init__(self, dnum, dcat, alpha=0.10):
        self.pre = make_preprocessor(dnum, dcat, for_regression=True)
        estimators = [
            ("rf", Pipeline([("pre", self.pre), ("m", RandomForestRegressor(n_estimators=600, min_samples_leaf=2, n_jobs=-1))])),
            ("hgb", Pipeline([("pre", self.pre), ("m", HistGradientBoostingRegressor(max_depth=None))])),
            ("ridge", Pipeline([("pre", self.pre), ("m", Ridge(alpha=1.0))])),
            ("mlp", Pipeline([("pre", self.pre), ("m", MLPRegressor(hidden_layer_sizes=(128,64), max_iter=800))])),
        ]
        if HAVE_XGB:
            estimators.append(("xgb", Pipeline([("pre", self.pre), ("m", xgb.XGBRegressor(
                n_estimators=1000, max_depth=7, subsample=0.9, colsample_bytree=0.9, tree_method="hist", n_jobs=-1))])))
        # Removed preprocessor from final_estimator
        self.stack = StackingRegressor(estimators=estimators, final_estimator=Ridge(alpha=1.0), passthrough=False, cv=5)
        self.fitted = False
        self.q_ = None
        self.alpha = alpha

    def fit(self, X, y):
        # Split-conformal calibration
        X_tr, X_cal, y_tr, y_cal = train_test_split(X, y, test_size=0.2, random_state=42)
        self.stack.fit(X_tr, y_tr)
        y_cal_pred = self.stack.predict(X_cal)
        resid = np.abs(y_cal - y_cal_pred)
        q = np.quantile(resid, 1-self.alpha)
        self.q_ = float(q)
        self.fitted = True
        return self

    def predict(self, X):
        return self.stack.predict(X)

    def predict_interval(self, X):
        assert self.fitted and self.q_ is not None
        yhat = self.stack.predict(X)
        return yhat - self.q_, yhat + self.q_

# ---------------------------- Metrics & Plots ---------------------------------
import matplotlib.pyplot as plt

def ece_score(y_true, prob, n_bins=15):
    bins = np.linspace(0.0, 1.0, n_bins+1)
    idx = np.digitize(prob, bins) - 1
    ece, total = 0.0, len(y_true)
    for b in range(n_bins):
        mask = idx == b
        if not np.any(mask): continue
        conf = prob[mask].mean()
        acc = (y_true[mask] == (prob[mask]>=0.5)).mean()
        ece += (mask.mean()) * abs(acc - conf)
    return float(ece)

def plot_bar(keys, values, title, ylabel, filename):
    plt.figure()
    plt.bar(keys, values)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, filename))
    plt.close()

# ------------------------------- Experiments ----------------------------------
def run_classification_suite(n=4000):
    specs = [
        ("linear", dict(kind="linear")),
        ("step", dict(kind="step")),
        ("sinusoidal", dict(kind="sinusoidal")),
        ("xor", dict(kind="xor")),
        ("mixed", dict(kind="mixed")),
    ]
    rows = []
    for tag, kwargs in specs:
        hdr(f"[Classification] Dataset = {tag}")
        X, y, dnum, dcat = gen_classification_dataset(n=n, d_num=8, d_cat=4, seed=42, **kwargs)
        Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=7, stratify=y)
        baselines = make_baselines_classif(dnum, dcat)
        rigel = make_rigel_classif(dnum, dcat)
        models = {**baselines, "RIGEL++": rigel}
        for name, model in models.items():
            t0 = time.time()
            model.fit(Xtr, ytr)
            proba = model.predict_proba(Xte)[:,1]
            pred = (proba>=0.5).astype(int)
            auc = roc_auc_score(yte, proba)
            acc = accuracy_score(yte, pred)
            ll = log_loss(yte, np.c_[1-proba, proba])
            brier = brier_score_loss(yte, proba)
            ece = ece_score(yte, proba)
            dt = time.time()-t0
            rows.append({"Task":"classification","Dataset":tag,"Model":name,"ROC_AUC":auc,"Accuracy":acc,"LogLoss":ll,"Brier":brier,"ECE":ece,"TrainTimeSec":dt})
            print(f"{name:10s} | AUC={auc:.3f} Acc={acc:.3f} LL={ll:.3f} Brier={brier:.3f} ECE={ece:.3f} Time={dt:.1f}s")
    df = pd.DataFrame(rows)
    df.to_csv(os.path.join(OUT_DIR, "classification_results.csv"), index=False)
    overall = df.groupby("Model", as_index=False).agg(ROC_AUC=("ROC_AUC","mean"), Accuracy=("Accuracy","mean"), ECE=("ECE","mean"))
    plot_bar(overall["Model"], overall["ROC_AUC"], "Classification — Mean ROC AUC", "Mean AUC", "cls_mean_auc.png")
    plot_bar(overall["Model"], overall["ECE"], "Classification — Mean ECE", "Mean ECE", "cls_mean_ece.png")
    return df

def run_regression_suite(n=4000):
    specs = [
        ("linear", dict(kind="linear")),
        ("step", dict(kind="step")),
        ("sinusoidal", dict(kind="sinusoidal")),
        ("hetero", dict(kind="hetero")),
        ("mixed", dict(kind="mixed")),
    ]
    rows = []
    for tag, kwargs in specs:
        hdr(f"[Regression] Dataset = {tag}")
        X, y, dnum, dcat = gen_regression_dataset(n=n, d_num=8, d_cat=4, seed=42, **kwargs)
        Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=7)
        baselines = make_baselines_reg(dnum, dcat)
        rigel = RigelRegressor(dnum, dcat, alpha=0.10)
        models = {**baselines, "RIGEL++": rigel}
        for name, model in models.items():
            t0=time.time()
            if name=="RIGEL++":
                model.fit(Xtr, ytr)
                yhat = model.predict(Xte)
                lo, hi = model.predict_interval(Xte)
                coverage = np.mean((yte>=lo) & (yte<=hi))
                width = np.mean(hi-lo)
            else:
                model.fit(Xtr, ytr)
                yhat = model.predict(Xte)
                coverage = np.nan
                width = np.nan
            rmse = math.sqrt(mean_squared_error(yte, yhat))
            mae = mean_absolute_error(yte, yhat)
            r2 = r2_score(yte, yhat)
            dt=time.time()-t0
            rows.append({"Task":"regression","Dataset":tag,"Model":name,"RMSE":rmse,"MAE":mae,"R2":r2,"PI90_Coverage":coverage,"PI90_Width":width,"TrainTimeSec":dt})
            print(f"{name:10s} | R2={r2:.3f} RMSE={rmse:.3f} MAE={mae:.3f} PI90_cov={coverage:.3f} width={width:.3f} Time={dt:.1f}s")
    df = pd.DataFrame(rows)
    df.to_csv(os.path.join(OUT_DIR, "regression_results.csv"), index=False)
    overall = df.groupby("Model", as_index=False).agg(R2=("R2","mean"))
    plot_bar(overall["Model"], overall["R2"], "Regression — Mean R^2", "Mean R^2", "reg_mean_r2.png")
    return df

def robustness_checks_cls(df_base: pd.DataFrame, n=2000):
    hdr("[Robustness] Classification stress test")
    X, y, dnum, dcat = gen_classification_dataset(n=n, d_num=8, d_cat=4, seed=24, kind="mixed")
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=9, stratify=y)
    Xte_stress = inject_missing_and_outliers(Xte, missing_frac=0.10, outlier_frac=0.05, outlier_scale=15.0)

    baselines = make_baselines_classif(dnum, dcat)
    rigel = make_rigel_classif(dnum, dcat)
    models = {**baselines, "RIGEL++": rigel}
    rows=[]
    for name, model in models.items():
        model.fit(Xtr, ytr)
        proba = model.predict_proba(Xte_stress)[:,1]
        auc = roc_auc_score(yte, proba)
        rows.append({"Model":name, "ROC_AUC_Stress":auc})
        print(f"{name:10s} | Stress AUC={auc:.3f}")
    df = pd.DataFrame(rows)
    df.to_csv(os.path.join(OUT_DIR, "classification_stress.csv"), index=False)
    return df

def robustness_checks_reg(df_base: pd.DataFrame, n=2000):
    hdr("[Robustness] Regression stress test")
    X, y, dnum, dcat = gen_regression_dataset(n=n, d_num=8, d_cat=4, seed=24, kind="hetero")
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=9)
    Xte_stress = inject_missing_and_outliers(Xte, missing_frac=0.10, outlier_frac=0.05, outlier_scale=15.0)

    baselines = make_baselines_reg(dnum, dcat)
    rigel = RigelRegressor(dnum, dcat, alpha=0.10)
    models = {**baselines, "RIGEL++": rigel}
    rows=[]
    for name, model in models.items():
        if name=="RIGEL++":
            model.fit(Xtr, ytr)
            yhat = model.predict(Xte_stress)
            lo, hi = model.predict_interval(Xte_stress)
            coverage = np.mean((yte>=lo) & (yte<=hi))
        else:
            model.fit(Xtr, ytr)
            yhat = model.predict(Xte_stress)
            coverage = np.nan
        rmse = math.sqrt(mean_squared_error(yte, yhat))
        rows.append({"Model":name, "RMSE_Stress":rmse, "PI90_Cov_Stress":coverage})
        print(f"{name:10s} | Stress RMSE={rmse:.3f} Cov={coverage}")
    df = pd.DataFrame(rows)
    df.to_csv(os.path.join(OUT_DIR, "regression_stress.csv"), index=False)
    return df

# ---------------------- Analytical Demonstrations ---------------------
ANALYTIC_NOTES = r"""
# Analytical Notes (RIGEL++)

**1) Stacking reduces expected generalization error (squared loss).**
Let base predictors \( f_1,\ldots,f_K \) and meta-learner \( g(w) = \sum_k w_k f_k \) with \( w \) fitted by ridge on OOF predictions. Since squared loss is convex and ridge yields the \(L_2\)-projection of \(y\) onto the span of \( \{f_k\} \), we have
\(\mathbb{E}[(y-g(w))^2] \le \min_k \mathbb{E}[(y-f_k)^2]\) whenever \(f_k\) lie in the hypothesis space and ridge includes the one-hot vectors as feasible \(w\) (proof by Pythagorean theorem in Hilbert space with penalty ensuring stability). Equality only if the best base is already in the span and orthogonal to others.

**2) Calibration by isotonic regression is risk-monotone.**
Given score \(s\) and labels \(y\), isotonic calibration finds a non-decreasing map \(h\) minimizing empirical log-loss among monotone transforms. It cannot worsen the reliability diagram’s monotonicity and typically reduces Brier/log-loss unless already perfectly calibrated.

**3) Split-conformal intervals guarantee marginal coverage.**
Under exchangeability of \((X_i,y_i)\), the split-conformal radius \(q = \text{Quantile}_{1-\alpha}(|y_i-\hat{y}_i|)\) computed on a calibration set satisfies
\(\mathbb{P}\{ y_{n+1} \in [\hat{y}(x_{n+1})-q,\ \hat{y}(x_{n+1})+q] \} \ge 1-\alpha\).
This holds for any underlying algorithm \(\hat{y}\); see Vovk et al. (2005).

**4) Robust preprocessing (quantile + Yeo–Johnson)** reduces sensitivity to outliers and skew, improving both trees and linear/meta learners, while one-hot on imputed categories keeps variance bounded for sparse high-cardinality features.
"""
with open(os.path.join(OUT_DIR, "ANALYTIC_NOTES.md"), "w", encoding="utf-8") as f:
    f.write(ANALYTIC_NOTES)

# ----------------------------- Validation Checks ------------------------------
def run_validations(cls_df: pd.DataFrame, reg_df: pd.DataFrame, cls_stress: pd.DataFrame, reg_stress: pd.DataFrame):
    checks = []

    # 1) Archivos generados
    expected_files = [
        "classification_results.csv", "regression_results.csv",
        "classification_stress.csv", "regression_stress.csv",
        "cls_mean_auc.png", "cls_mean_ece.png", "reg_mean_r2.png", "ANALYTIC_NOTES.md"
    ]
    files_present = all(os.path.exists(os.path.join(OUT_DIR, fn)) for fn in expected_files)
    checks.append(("artifacts_exist", files_present, expected_files))

    # 2) RIGEL++ mejora AUC medio vs cada baseline (tolerancia ligera)
    rigel_auc = cls_df[cls_df["Model"]=="RIGEL++"]["ROC_AUC"].mean()
    base_auc = cls_df[cls_df["Model"]!="RIGEL++"].groupby("Model")["ROC_AUC"].mean()
    rigel_beats_all_auc = bool((rigel_auc >= (base_auc.max() - 1e-6)))
    checks.append(("rigel_auc_ge_best_baseline", rigel_beats_all_auc, float(rigel_auc), float(base_auc.max())))

    # 3) ECE de RIGEL++ <= promedio de ECE baselines
    rigel_ece = cls_df[cls_df["Model"]=="RIGEL++"]["ECE"].mean()
    base_ece_mean = cls_df[cls_df["Model"]!="RIGEL++"]["ECE"].mean()
    ece_ok = bool(rigel_ece <= base_ece_mean + 1e-6)
    checks.append(("rigel_ece_le_baseline_mean", ece_ok, float(rigel_ece), float(base_ece_mean)))

    # 4) Cobertura conformal ~ 0.90 ± 0.05 (promedio)
    rigel_cov = reg_df[reg_df["Model"]=="RIGEL++"]["PI90_Coverage"].mean()
    cov_ok = bool((rigel_cov >= 0.85) and (rigel_cov <= 0.95))
    checks.append(("rigel_pi90_coverage_approx_0.90", cov_ok, float(rigel_cov)))

    # 5) RIGEL++ R² medio ≥ mejor baseline R² medio - 0.01
    rigel_r2 = reg_df[reg_df["Model"]=="RIGEL++"]["R2"].mean()
    base_r2_max = reg_df[reg_df["Model"]!="RIGEL++"].groupby("Model")["R2"].mean().max()
    r2_ok = bool(rigel_r2 >= base_r2_max - 0.01)
    checks.append(("rigel_r2_ge_best_baseline_minus_0.01", r2_ok, float(rigel_r2), float(base_r2_max)))

    # 6) Stress: AUC de RIGEL++ ≥ mediana de baselines en stress
    rigel_auc_stress = cls_stress[cls_stress["Model"]=="RIGEL++"]["ROC_AUC_Stress"].mean()
    base_auc_stress_median = cls_stress[cls_stress["Model"]!="RIGEL++"]["ROC_AUC_Stress"].median()
    stress_auc_ok = bool(rigel_auc_stress >= base_auc_stress_median - 1e-6)
    checks.append(("rigel_stress_auc_ge_baseline_median", stress_auc_ok, float(rigel_auc_stress), float(base_auc_stress_median)))

    # 7) Stress: RMSE de RIGEL++ ≤ mediana de baselines en stress + 0.02
    rigel_rmse_stress = reg_stress[reg_stress["Model"]=="RIGEL++"]["RMSE_Stress"].mean()
    base_rmse_stress_median = reg_stress[reg_stress["Model"]!="RIGEL++"]["RMSE_Stress"].median()
    stress_rmse_ok = bool(rigel_rmse_stress <= base_rmse_stress_median + 0.02)
    checks.append(("rigel_stress_rmse_le_baseline_median_plus_0.02", stress_rmse_ok, float(rigel_rmse_stress), float(base_rmse_stress_median)))

    # 8) XGBoost detectado (solo informativo, no es obligatorio)
    checks.append(("xgboost_available", HAVE_XGB, None))

    # Print scoreboard
    hdr("SCOREBOARD — Validations (booleans + valores)")
    for item in checks:
        name = item[0]
        ok = item[1]
        vals = item[2:] if len(item)>2 else ()
        print(f"{name:40s}: {1 if ok else 0} | values = {vals}")
    return checks

# ----------------------------------- Main ------------------------------------
def main():
    hdr("RIGEL")
    env_report()

    cls_df = run_classification_suite(n=3500)
    reg_df = run_regression_suite(n=3500)
    cls_stress = robustness_checks_cls(cls_df, n=2500)
    reg_stress = robustness_checks_reg(reg_df, n=2500)

    # Summaries
    hdr("Summary — Toplines")
    print("Classification (mean across datasets):")
    print(cls_df.groupby("Model").agg(ROC_AUC=("ROC_AUC","mean"),
                                      Accuracy=("Accuracy","mean"),
                                      ECE=("ECE","mean"),
                                      TrainTime=("TrainTimeSec","mean")).sort_values("ROC_AUC", ascending=False))

    print("\nRegression (mean across datasets):")
    print(reg_df.groupby("Model").agg(R2=("R2","mean"),
                                      RMSE=("RMSE","mean"),
                                      MAE=("MAE","mean"),
                                      TrainTime=("TrainTimeSec","mean")).sort_values("R2", ascending=False))

    print("\nRobustness — Classification (Stress AUC):")
    print(cls_stress.sort_values("ROC_AUC_Stress", ascending=False))

    print("\nRobustness — Regression (Stress RMSE):")
    print(reg_stress.sort_values("RMSE_Stress"))

    checks = run_validations(cls_df, reg_df, cls_stress, reg_stress)

    print(f"\nArtifacts saved in: {OUT_DIR}")
    print("Files:")
    for fn in sorted(os.listdir(OUT_DIR)):
        print(" -", fn)

if __name__ == "__main__":
    main()


RIGEL++ — Full Benchmark (Colab VM dependent)
Environment: {'python': '3.12.11', 'numpy': '2.0.2', 'pandas': '2.2.2', 'matplotlib': '3.10.0', 'sklearn': '1.6.1'}

[Classification] Dataset = linear
LogReg     | AUC=0.918 Acc=0.842 LL=0.347 Brier=0.109 ECE=0.513 Time=0.3s
RF         | AUC=0.900 Acc=0.729 LL=0.524 Brier=0.172 ECE=0.427 Time=2.1s
HGB        | AUC=0.893 Acc=0.817 LL=0.400 Brier=0.128 ECE=0.520 Time=1.6s
MLP        | AUC=0.878 Acc=0.817 LL=1.300 Brier=0.163 ECE=0.622 Time=6.8s
XGB        | AUC=0.888 Acc=0.815 LL=0.617 Brier=0.150 ECE=0.583 Time=3.0s
RIGEL++    | AUC=0.920 Acc=0.847 LL=0.344 Brier=0.108 ECE=0.528 Time=254.4s

[Classification] Dataset = step
LogReg     | AUC=0.862 Acc=0.784 LL=0.440 Brier=0.144 ECE=0.440 Time=0.1s
RF         | AUC=0.869 Acc=0.728 LL=0.535 Brier=0.177 ECE=0.396 Time=1.6s
HGB        | AUC=0.877 Acc=0.801 LL=0.420 Brier=0.136 ECE=0.484 Time=1.8s
MLP        | AUC=0.824 Acc=0.761 LL=1.365 Brier=0.211 ECE=0.604 Time=9.4s
XGB        | AUC=0.872 Acc=